In [1]:
# Load embeddings
from pickle import load
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

embeds = load(open("jepa_encodings.pkl", "rb"))

# Break into categories
clothing_embed = {k:v for k, v in embeds.items() if k.endswith("c.png")}
clothing_labels = [k.split("-")[0].split("_") for k in clothing_embed.keys()]
clothing_labels = [[lab[0], lab[1], lab[2][:-1]] if lab[2].endswith("s") else lab for lab in clothing_labels]
clothing_embed = np.array(list(clothing_embed.values()))

full_body_embed = {k:v for k, v in embeds.items() if k.endswith("fb.png")}
full_body_filenames = [k for k in full_body_embed.keys()]
full_body_labels = [k.split("-")[0].split("_") for k in full_body_embed.keys()]
full_body_labels = [[lab[0], lab[1], lab[2][:-1]] if lab[2].endswith("s") else lab for lab in full_body_labels]
full_body_embed = np.array(list(full_body_embed.values()))

In [2]:
# Preprocess embedings
clothing_genders = np.zeros((clothing_embed.shape[0], 2))
clothing_genders[np.array([{"mens": 0, "womens": 1}[lab[1]] for lab in clothing_labels])] = 1

clothing_type = np.zeros((clothing_embed.shape[0], 6))
clothing_type[np.array([
    {"shirt": 0, "top": 1, "sweater": 2, "pant": 3, "skirt": 4, "short": 5}[lab[2]]
    for lab in clothing_labels
])] = 1

In [3]:
# Train model
import tensorflow as tf
from copy import deepcopy

class triplet_model(tf.keras.Model):
    def __init__(self, clothing_layers, outfit_layers, margin=0, trainable=True,):
        super().__init__(trainable=trainable)
        self.clothing_model = tf.keras.Sequential(clothing_layers)
        self.outfit_model = tf.keras.Sequential(outfit_layers)
        self.margin = margin

    def compile(self, **kwargs):
        self.clothing_model.compile(**kwargs)
        self.outfit_model.compile(**kwargs)
        super().compile(**kwargs)
    
    # https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit#supporting_sample_weight_class_weight
    # https://en.wikipedia.org/wiki/Triplet_loss
    def train_step(self, data):
        x, x_positive, x_negative = data

        with tf.GradientTape() as tape:
            y_anchor = self.clothing_model(x)
            y_positive = self.outfit_model(x_positive)
            y_negative = self.outfit_model(x_negative)
            
            dist_positive = tf.math.sqrt(tf.math.reduce_sum(tf.math.square(y_anchor - y_positive), axis=-1))
            dist_negative = tf.math.sqrt(tf.math.reduce_sum(tf.math.square(tf.expand_dims(y_anchor, axis=1) - tf.expand_dims(y_positive, axis=0)), axis=-1))
            
            loss = tf.math.maximum(tf.expand_dims(dist_positive, axis=1) - dist_negative + self.margin, 0)
            # loss = tf.expand_dims(dist_positive, axis=1) - dist_negative
            # loss = tf.math.reduce_max(loss, axis=1)
            loss = tf.math.reduce_mean(loss)

        # Compute gradients
        trainable_vars = self.clothing_model.trainable_variables + self.outfit_model.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics.
        # Metrics are configured in `compile()`.
        dist_negative *= (1 - tf.eye(tf.shape(x)[0], tf.shape(x)[0]))
        dist_negative += tf.eye(tf.shape(x)[0], tf.shape(x)[0]) * np.inf
        dist_negative = tf.math.reduce_min(dist_negative, axis=1)
        for metric in self.metrics:
            if metric.name == "loss":
                metric.update_state(loss)
            else:
                metric.update_state(tf.ones((tf.shape(x)[0], 1)), tf.expand_dims(tf.cast(dist_positive < dist_negative, float), axis=-1))

        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        x, x_positive, x_negative = data

        y_anchor = self.clothing_model(x)
        y_positive = self.outfit_model(x_positive)
        y_negative = self.outfit_model(x_negative)
        
        dist_positive = tf.math.sqrt(tf.math.reduce_sum(tf.math.square(y_anchor - y_positive), axis=-1))
        dist_negative = tf.math.sqrt(tf.math.reduce_sum(tf.math.square(tf.expand_dims(y_anchor, axis=1) - tf.expand_dims(y_positive, axis=0)), axis=-1))
            
        loss = tf.math.maximum(tf.expand_dims(dist_positive, axis=1) - dist_negative + self.margin, 0)
        # loss = tf.expand_dims(dist_positive, axis=1) - dist_negative
        # loss = tf.math.reduce_max(loss, axis=1)
        loss = tf.math.reduce_mean(loss)


        # Update the metrics.
        # Metrics are configured in `compile()`.
        dist_negative *= (1 - tf.eye(tf.shape(x)[0], tf.shape(x)[0]))
        dist_negative += tf.eye(tf.shape(x)[0], tf.shape(x)[0]) * np.inf
        dist_negative = tf.math.reduce_min(dist_negative, axis=1)
        for metric in self.metrics:
            if metric.name == "loss":
                metric.update_state(loss)
            else:
                metric.update_state(tf.ones((tf.shape(x)[0], 1)), tf.expand_dims(tf.cast(dist_positive < dist_negative, float), axis=-1))

        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}


def triplet_sample_generator(validation=False):
    clothing = np.concatenate([clothing_embed, clothing_genders, clothing_type], axis=-1)
    outfit = np.concatenate([full_body_embed, clothing_genders, clothing_type], axis=-1)
    
    if validation:
        inds = np.arange(clothing_embed.shape[0])[np.arange(clothing_embed.shape[0]) % 10 == 0]
    else:
        inds = np.arange(clothing_embed.shape[0])[np.arange(clothing_embed.shape[0]) % 10 != 0]

    np.random.shuffle(inds)

    for i in inds:
        neg_inds = inds[inds != i]
        np.random.shuffle(neg_inds)
        yield (clothing[i], outfit[i], outfit[neg_inds[0]])

ds_train = tf.data.Dataset.from_generator(triplet_sample_generator, output_types=(float, float, float))
ds_valid = tf.data.Dataset.from_generator(lambda: triplet_sample_generator(validation=True), output_types=(float, float, float))

model = triplet_model(
    [
        tf.keras.layers.Input((clothing_embed.shape[1] + 2 + 6,)),
        tf.keras.layers.UnitNormalization(),
        tf.keras.layers.Dense(clothing_embed.shape[1] * 8, activation="elu"),
        tf.keras.layers.Dropout(.1),
        tf.keras.layers.Dense(clothing_embed.shape[1] * 12, activation="elu"),
        tf.keras.layers.Dropout(.1),
        tf.keras.layers.Dense(clothing_embed.shape[1] * 12, activation="elu"),
        tf.keras.layers.Dropout(.1),
        tf.keras.layers.Dense(clothing_embed.shape[1] * 8, activation="elu"),
        tf.keras.layers.Dropout(.1),
        tf.keras.layers.Dense(128),
        tf.keras.layers.BatchNormalization(),
    ], [
        tf.keras.layers.Input((full_body_embed.shape[1] + 2 + 6,)),
        tf.keras.layers.UnitNormalization(),
        tf.keras.layers.Dense(full_body_embed.shape[1] * 8, activation="elu"),
        tf.keras.layers.Dropout(.1),
        tf.keras.layers.Dense(full_body_embed.shape[1] * 12, activation="elu"),
        tf.keras.layers.Dropout(.1),
        tf.keras.layers.Dense(full_body_embed.shape[1] * 12, activation="elu"),
        tf.keras.layers.Dropout(.1),
        tf.keras.layers.Dense(full_body_embed.shape[1] * 8, activation="elu"),
        tf.keras.layers.Dropout(.1),
        tf.keras.layers.Dense(128),
        tf.keras.layers.BatchNormalization(),
    ],
    margin = 64.
)

model.compile(
    optimizer="adam",
    run_eagerly=True,
    metrics=["binary_crossentropy", "binary_accuracy"]
)

model.fit(
    ds_train.batch(256),
    epochs=500,
    validation_data=ds_valid.batch(256),
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=100, restore_best_weights=True)],
)   

2024-05-01 14:25:23.101797: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-01 14:25:23.147517: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-01 14:25:23.867809: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Instructions for updating:
Use output_signature instead
Epoch 1/500


2024-05-01 14:25:24.688676: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22253 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:18:00.0, compute capability: 8.9
2024-05-01 14:25:24.689237: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22446 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:af:00.0, compute capability: 8.6


      3/Unknown 3s 224ms/step - binary_accuracy: 0.0000e+00 - binary_crossentropy: 16.1181 - loss: 64.0197

2024-05-01 14:25:27.834270: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/home/ian/miniconda3/envs/tf/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
2024-05-01 14:25:27.931733: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

2024-05-01 14:28:15.506471: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-01 14:28:15.641272: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 344ms/step - binary_accuracy: 0.0000e+00 - binary_crossentropy: 16.1181 - loss: 49.7743 - val_binary_accuracy: 0.0000e+00 - val_binary_crossentropy: 16.1181 - val_loss: 70.1593
Epoch 140/500


2024-05-01 14:28:16.676857: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-01 14:28:16.822711: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 333ms/step - binary_accuracy: 0.0000e+00 - binary_crossentropy: 16.1181 - loss: 49.4447 - val_binary_accuracy: 0.0000e+00 - val_binary_crossentropy: 16.1181 - val_loss: 73.0749


2024-05-01 14:28:17.861448: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-01 14:28:18.001222: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [4]:
# Test model
converted_clothing_embeds = model.clothing_model(np.concatenate([clothing_embed, clothing_genders, clothing_type], axis=-1))
converted_clothing_embeds = tf.expand_dims(converted_clothing_embeds, axis=1)
print(converted_clothing_embeds)

converted_outfit_embeds = model.outfit_model(np.concatenate([full_body_embed, clothing_genders, clothing_type], axis=-1))
converted_outfit_embeds = tf.expand_dims(converted_outfit_embeds, axis=0)
print(converted_outfit_embeds)

distances = tf.math.sqrt(tf.math.reduce_sum(tf.math.square(converted_outfit_embeds - converted_clothing_embeds), axis=-1))
best_samples = tf.math.top_k(tf.math.reciprocal(distances), k=5).indices

best_samples

tf.Tensor(
[[[ 140141.55  -762666.94  -209075.89  ...   73049.234  -99083.03
    203420.33 ]]

 [[ 142534.23  -776593.2   -213096.62  ...   74174.7   -100559.16
    207479.67 ]]

 [[ 134893.1   -734677.25  -201548.03  ...   70227.484  -95214.45
    196193.36 ]]

 ...

 [[ 144699.86  -788032.1   -216153.23  ...   75353.086 -102182.805
    210393.42 ]]

 [[ 139978.95  -762235.56  -209064.36  ...   72899.016  -98852.92
    203486.84 ]]

 [[ 141531.08  -770184.56  -211117.95  ...   73784.89  -100086.03
    205401.22 ]]], shape=(624, 1, 128), dtype=float32)
tf.Tensor(
[[[-152.41367    257.17102    227.48766   ...   25.051975  -275.66675
      6.533869 ]
  [-153.67162    260.61523    229.89064   ...   25.217714  -277.72733
      5.9742184]
  [-153.32184    259.60773    229.11516   ...   25.139103  -276.90326
      6.456886 ]
  ...
  [-152.98022    259.7248     229.04141   ...   25.598091  -276.61215
      5.863697 ]
  [-153.91907    259.89374    229.83847   ...   25.352001  -277.60046
      

<tf.Tensor: shape=(624, 5), dtype=int32, numpy=
array([[154, 297,  70, 484, 275],
       [154, 297,  70, 484, 275],
       [154, 297,  70, 484, 275],
       ...,
       [154, 297,  70, 484, 275],
       [154, 297,  70, 484, 275],
       [154, 297,  70, 484, 275]], dtype=int32)>

In [5]:
full_body_filenames[154]

'jcrew_womens_tops-BY693-WT0002-fb.png'

In [6]:
model.outfit_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ unit_normalization_1            │ (None, 1288)           │             0 │
│ (UnitNormalization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10240)          │    13,199,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 10240)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 15360)          │   157,301,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 15360)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 15360)          │   235,944,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 15360)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 10240)          │   157,296,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 10240)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 128)            │     1,310,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 565,054,080 (2.10 GB)

 Trainable params: 565,053,824 (2.10 GB)

 Non-trainable params: 256 (1.00 KB)